In [1]:
## S1 - 1 Point MD BER Behavior

In [3]:
from random import seed, shuffle
from tqdm import tqdm
import sys
sys.path.insert(1, '/home/preuer/OneDrive/Studium/Master/Semester/Sem_4/Master Thesis/fairness_triangle/tools') # so i can import tools
from gen_synth_data import *
from plot_helper import *
from corrupt_labels import *
from calc_metrics import *
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
#Configuration to make plots nicer
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
np.random.seed(0)  # Set seed for reproducibility
flip_prob = 0.2 # Probability of flipping each value   0, 0.1 0.2 ... 1
disc_factor = 0.5
lmd = -0.5


X, Y, Y_sen = generate_synthetic_data(False, n_samples=2000, disc_factor=disc_factor)    #n_samples are per gaussian distributions, we have two hence total_n = 2*n_samples

Y_corrupted = add_bin_noise(Y, flip_prob)
Y_sen_corrupted = add_bin_noise(Y_sen, flip_prob)

#Devide Set into training and test set
split_index = int(0.7 * len(X))

# Split the points into training and testing sets (80/20)
X_train, X_test = X[:split_index], X[split_index:]
Y_train, Y_test = Y[:split_index], Y[split_index:]
Y_sen_train, Y_sen_test = Y_sen[:split_index], Y_sen[split_index:]
Y_corr_train, Y_corr_test = Y_corrupted[:split_index], Y_corrupted[split_index:]
Y_sen_corr_train, Y_sen_corr_test = Y_sen_corrupted[:split_index], Y_sen_corrupted[split_index:]


p_reg, p_reg_cor, f_reg, f_reg_cor = LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()
p_reg.fit(X_train, Y_train)
p_reg_cor.fit(X_train, Y_corr_train)
f_reg.fit(X_train, Y_sen_train)
f_reg_cor.fit(X_train, Y_sen_corr_train)



BER_list = []
MD_list = []
DI_list = []

BER_list_corr = []
MD_list_corr = []
DI_list_corr = []

# Cost sensitivity of classes
c, c_bar = 0.5, 0.5


In [6]:
def calc_perf_fair(lmd, x, y, y_sen, y_corr, y_sen_corr):
    c, c_bar = 0.5, 0.5
    
    #predict point using f_reg and normal targets
    s = p_reg.predict_proba([x])[:, 1] - c - lmd * (f_reg.predict_proba([x])[:, 1] - c_bar)
    # Apply step function H_a(z) with a = 0
    y_pred = np.where(s > 0, 1, 0)
        
    BER = calc_BER(y_pred, y)
    MD = calc_MD(y_pred, y_sen, False)
    DI = calc_DI(y_pred, y_sen, False)

    #predict point using f_reg and corrupted targets
    s = p_reg.predict_proba([x])[:, 1] - c - lmd * (f_reg.predict_proba([x])[:, 1] - c_bar)
    # Apply step function H_a(z) with a = 0
    y_pred = np.where(s > 0, 1, 0)

    BER_corr = calc_BER(y_pred, y_corr)
    MD_corr = calc_MD(y_pred, y_sen_corr, False)
    DI_corr = calc_DI(y_pred, y_sen_corr, False)
    
    return BER, MD, DI, BER_corr, MD_corr, DI_corr

In [9]:
i=9
BER, MD, DI, BER_corr, MD_corr, DI_corr = calc_perf_fair(0, X_test[i], Y_test[i], Y_sen_test[i], Y_corr_test[i], Y_sen_corr_test[i])
print("BER: " + str(BER) + "\n"  "BER_corr:" + str(BER_corr) +  "MD: " + str(MD) + "\n" + "MD_corr: " + str(MD_corr) + "\n" + "DI: " + str(DI) + "\n" + "DI_corr: " + str(DI_corr))
print()
print(print(X_test[i], Y_test[i], Y_corr_test[i], Y_sen_test[i], Y_sen_corr_test[i]))

BER: 0.0
BER_corr:0.0MD: 0.0
MD_corr: -1.0
DI: 1.0
DI_corr: 0.0

[3.98928517 0.6635216 ] 1.0 1.0 0.0 1.0
None
